In [1]:
import torch
import torch.nn as nn 
from torch.nn import functional as F



if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")



tensor([1.], device='mps:0')


In [2]:
### paramters
block_size =64
batch_size =128
max_iterations = 1000
learning_rate =3e-4
eval_iters = 250
n_embd = 384
n_head = 4
n_layer =4
dropout = 0.2

In [3]:
char =""

with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))
vocab_size = len(chars)

In [4]:
### character tokenizer from scratch 
string_to_int ={ch:i for i, ch in enumerate(chars)} ## mapping aka dicts
int_to_string ={i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])
## using pytorch tensors to encode the entire text
data =torch.tensor(encode(text), dtype= torch.long)

In [5]:
n =int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split =='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[d:d+block_size] for d in ix ])
    y = torch.stack([data[d+1:d+block_size+1] for d in ix])
    x,y = x.to(mps_device),y.to(mps_device)
    return x,y

x,y = get_batch('train')
print(f"inputs:\n {x}")
print(f"traget:\n {y}")

inputs:
 tensor([[50, 67, 64,  ..., 65, 54, 50],
        [53,  1, 64,  ...,  1, 58, 63],
        [56,  1, 70,  ..., 72, 50, 68],
        ...,
        [53,  1, 50,  ..., 58, 54, 53],
        [53,  1, 57,  ..., 57, 54, 62],
        [ 6,  1, 69,  ..., 55, 70, 69]], device='mps:0')
traget:
 tensor([[67, 64, 70,  ..., 54, 50, 67],
        [ 1, 64, 63,  ..., 58, 63, 66],
        [ 1, 70, 65,  ..., 50, 68,  1],
        ...,
        [ 1, 50, 63,  ..., 54, 53,  1],
        [ 1, 57, 50,  ..., 54, 62,  1],
        [ 1, 69, 57,  ..., 70, 69, 70]], device='mps:0')


In [6]:
class Head(nn.Module):
    """one head self attention"""
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd,head_size,bias=False)
        self.query = nn.Linear(n_embd,head_size, bias=False)
        self.value = nn.Linear(n_embd,head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size))) ## register in model state can be removed but will take longer to train

        self.dropout= nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)  
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T) ### scaling mathhhhh
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [7]:
class MultiHeadAttention(nn.Module):
    '''multiple heads of self-attenton parralel '''
    def __init__(self, num_heads,head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) ## heads in parallel 
        self.proj = nn.Linear(head_size * num_heads, n_embd) ## project head size ## changeable to not throw dimintionality errors
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads], dim = -1) ## 
        out = self.dropout(self.proj(out))
        return out

In [8]:
class FeedForward(nn.Module):
    '''linear layer followed by a non-linearity'''

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd,4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd,n_embd),
            nn.Dropout(dropout),

        )
    def forward(self,x):
        return self.net(x)


In [9]:
class Block(nn.Module):
    """ Transformer block: communications followed by computation """

    def __init__(self,n_embd,n_head):
        ## n_embd: embedding dimnsion , N_head : the number of heads we'd like
        super().__init__()
        head_size  =n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size) ## self attention
        self.ffwd = FeedForward(n_embd) 
        self.ln1 = nn.LayerNorm(n_embd) ## post norm 
        self.ln2 = nn.LayerNorm(n_embd)
    def forward (self,x):
        y = self.sa(x)
        x = self.ln1(x+y)
        y= self.ffwd(x)
        x = self.ln2(x + y)
        return x

In [10]:
### neural network 
class GPTModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        ### std used by professinoals mostly understand it as much as you can
        ## basically is it weight and bias initalizing using std 
        ## for trainig to converge better
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean = 0.0, std =0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)



    def forward(self, index, target= None):
        B, T = index.shape

        ## idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # token_emedding_table
        pos_emb = self.position_embedding_table(torch.arange(T, device =mps_device)) ## how long is T then give a number here incdies 8
        x = tok_emb + pos_emb ## adding them together 
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        

        if target is None:
            loss = None
        else:
            B, T, C =logits.shape ## batch / time / 
            logits = logits.view(B*T, C)
            target = target.view(B*T) ## reshaping bc cross entropy 
            loss = F.cross_entropy(logits, target)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        ## starting here the index is (B,T) array of indices 
        for _ in range(max_new_tokens):
            ## getting predictions
            logits, loss = self.forward(index)
            logits = logits[:,-1,:] ## to get previous step aw last since bigram model
            probs = F.softmax(logits, dim=-1) ## softmax function
            index_next = torch.multinomial(probs, num_samples=1)#sample from distribution(B,1)
            index = torch.cat((index, index_next), dim=1) ## adding the prev and next index matrix
        return index
    
model = GPTModel(vocab_size)
m = model.to(mps_device)

# context = torch.zeros((1,1), dtype=torch.long, device=mps_device)
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)


In [11]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split]= losses.mean()
    model.train()
    return out


In [12]:
optimizer  = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iterations):
    if iter % eval_iters==0:
        losses = estimate_loss()
        print(f"step:{iter},train loss{losses['train']:.3f}, val loss: {losses['val']:.3f}")

    xb, yb = get_batch('train')## batch data sample 

    ## loss eval
    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none= True)
    loss.backward()
    optimizer.step()
print(loss.item())

/Users/abdullah/miniforge3/envs/gog/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


step:0,train loss4.426, val loss: 4.425
step:250,train loss1.807, val loss: 1.850
step:500,train loss1.487, val loss: 1.583
step:750,train loss1.340, val loss: 1.482
1.3297969102859497


In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=mps_device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)